# Adrien HANS & Tanguy JEANNEAU

# Travaux pratiques - Optimisation non linéaire avec contraintes : 

In [1]:
import matplotlib.pyplot as plt
import scipy.optimize as opt
import numpy as np

## Partie 1 - Méthode de pénalité extérieure : 

**Soit le problème suivant :** 

\begin{align*}
Min \hspace{0.1cm} f(x_1,x_2,x_3) = -x_1-x_2+x_3
\\Sous\hspace{0.1cm} contrainte
\\x_1^3+x_3 \leq 1 
\\x_1^2+x_2^2+x_3^2 \leq1 
\\0 \leq x_3 \leq 1
\end{align*}


**Problème général avec la méthode de pénalité :**
\begin{align*}
min f(x)
\end{align*}
Sous contraintes :
\begin{align*}
g_i(x)\leq,i=1,...,m
\end{align*}
Problème pénalisé : 
\begin{align*}
min_x f(x)+(1/2\rho) ||max(0,g(x)||^2
\end{align*}
Condition d'optimalité : 
\begin{align*}
P(x,\rho)=\nabla f(x) +(1/\rho)\sum_i max(0,g_i(x))\nabla g_i(x)=0
\end{align*}

**Ici on a :**
\begin{align*}
f(x)=-x_1-x_2+x_3\\
g_1(x)=x_1^3+x_3-1\\
g_2(x)=x_1^2+x_2^2+x_3^2-1\\
g_3(x)=-x_3\\
g_4(x)=x_3-1\\
\end{align*}

### **1)**On détermine la solution optimale de ce problème en utilisant une méthode de pénalité quadratique: 
On définit dans un premier temps l'ensemble des fonctions du problème ainsi que leur gradients.

In [2]:
def f(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    return -x1-x2+x3

def gradf(x):
    return np.array([-1,-1,1])

def g1(x):
    return x[0]**3+x[2]-1
def gradg1(x):
    return np.array([3*x[0]**2,0,1])

def g2(x):
    return x[0]**2 + x[1]**2 + x[2]**2 -1 
def gradg2(x):
    return np.array([2*x[0],2*x[1],2*x[2]])

def g3(x):
    return -x[2]
def gradg3(x):
    return np.array([0,0,-1])

def g4(x):
    return x[2]-1
def gradg4(x):
    return np.array([0,0,1])

On applique la méthode de pénalité à ce problème. 

In [3]:
#Données : 
epsilon = 0.00001
beta=0.6
x=np.array([2,1,1])
rho=0.7
tolerance=0.01
k=1 

def P(x):
    p1=np.maximum(0,g1(x))*gradg1(x)
    p2=np.maximum(0,g2(x))*gradg2(x)
    p3=np.maximum(0,g3(x))*gradg3(x)
    p4=np.maximum(0,g4(x))*gradg4(x)
    Sum=p1+p2+p3+p4
    P=gradf(x)+(1/rho)*Sum
    return np.linalg.norm(P)
#Algorithme méthode de pénalité :
while rho>epsilon:
    x=x
    while P(x)>tolerance:
        
        """
        Nous voulions dans un premier temps résoudre la minimisation de P de façon manuelle grâce à ce type de manoeuvres :
        
        
        
        On détermine une direction admissible :
        direction d du gradient:
        d=-np.gradient(P(x))
        (On pourrait aussi prendre le direction de Newton par exemple)
        On détermine un pas admissible :
        alpha=opt.line_search(P,gradP,x,d)
        On met à jour x : 
        x=x+alpha[0]*d
        
        
        Cependant, ces essais n'ayant pas abouties, 
        on résout la minimisation de P tout simplement avec la fonction minimize
        de la librairie scipy.optmize : 
        """
        
        x=opt.minimize(P,x).x        
    x=x
    rho=beta*rho
    k+=1
    
    
print('x : ', x)
print('valeur de f en x  : ' ,f(x))

x :  [ 7.09513113e-01  7.04699928e-01 -1.53188102e-05]
valeur de f en x  :  -1.4142283590960203


**Nous avons donc trouvé une valeur de x solution de ce problème par la méthode de pénalité.**

Après plusieurs essais, nous avons choisis des valeurs initiales pour $\epsilon$, $\beta$, $\rho$, et de tolérance pour que l'algorithme converge bien, sans être trop rapide dans la convergence pour avoir de bonnes valeurs finales.

#### Vérification de la satisfaction des contraintes : 

In [4]:
ineq1=x[0]**3+x[2]
ineq2=x[0]**2+x[1]**2+x[2]**2

Première inégalité : 
$x_1^3+x_3 \leq 1$ 
Or ici : $x_1^3+x_3=$ :

In [5]:
print(ineq1)

0.35715986605614586


Deuxième inégalité : 
$x_1^2+x_2^2+x_3^2 \leq1$ Or ici : $x_1^2+x_2^2+x_3^2=$ :

In [6]:
print(ineq2)

1.0000108452654577


Troisième inégalité :
$0 \leq x_3 \leq 1$ Or ici, $x_3=$ : 

In [7]:
print(x[2])

-1.5318810241796736e-05


**On vérifie donc bien dans l'ensemble qu'il s'agit d'une méthode éxtérieure, c'est à dire que certaines contraintes sont "presque" satisfaites.**

### 2) Estimation des valeurs des multiplicateurs de Lagrange : 
Etant donné la nature de la formule issue des conditions de Karush-Kuhn-Tucker et la condition d'optimalité, on peut approcher chaque multiplicateur de Lagrange $\lambda_i$ par la formule : 
\begin{align*}
\lambda_i = \frac{max(0,g_i(x^*))}{\rho}
\end{align*}
Ainsi on pour multiplicateurs de Lagrange : 

In [8]:
lamb1=np.maximum(0,g1(x))/rho
lamb2=np.maximum(0,g2(x))/rho
lamb3=np.maximum(0,g3(x))/rho
lamb4=np.maximum(0,g4(x))/rho

print('1er Multiplicateur de Lagrange : ', lamb1)
print('2eme Multiplicateur de Lagrange : ', lamb2)
print('3eme Multiplicateur de Lagrange : ', lamb3)
print('4eme Multiplicateur de Lagrange : ', lamb4)

1er Multiplicateur de Lagrange :  0.0
2eme Multiplicateur de Lagrange :  1.1771034651473258
3eme Multiplicateur de Lagrange :  1.6626448368548388
4eme Multiplicateur de Lagrange :  0.0


#### Contraintes actives :
Une contrainte est active si son multiplicateur de Lagrange associé est non-nul. 

Ainsi, étant donné les valeurs des multiplicateurs de Lagrange ici, les contraintes associées aux valeurs non nulles des multiplicateurs de Lagrange sont actives, c'est-à-dire les contraintes associées à $\lambda_2$ et à $\lambda_3$, donc les contraintes suivantes : 

\begin{align*}
\\x_1^2+x_2^2+x_3^2 \leq1 
\\0 \leq x_3
\end{align*}

### 3) Obtient-on un col ? 

Calcul de la fonction de Lagrange : 

In [9]:
def func_Lag1(f,x,lamb1,lamb2,lamb3,lamb4,g1,g2,g3,g4):   
    p1=lamb1*g1(x)
    p2=lamb2*g2(x)
    p3=lamb3*g3(x)
    p4=lamb4*g4(x)
    Sum=p1+p2+p3+p4
    L=f(x)-Sum
    return L
L=func_Lag1(f,x,lamb1,lamb2,lamb3,lamb4,g1,g2,g3,g4)
print('Valeur de la fonction de Lagrange : ', L)

Valeur de la fonction de Lagrange :  -1.4142665948363262


**Par définition, on sait qu'on obtient un col si :**
\begin{align*}
L(x^*,\lambda)\leq L(x^*,\lambda^*) \leq L(x,\lambda^*) 
\end{align*}

Ceci étant difficile à démontrer étant donné la nature des objets que l'on considère, on va plutôt vérifier si on obtient un col via la caractérisation de celui-ci, c'est-à-dire vérifier que : 

**1)**
\begin{align*}
L(x^*,\lambda^*)=min_xL(x,\lambda^*)
\end{align*}
**2)**
\begin{align*}
g_i(x^*)\geq0, \hspace{0.5cm} i=1,..,m
\end{align*}
**3)**
\begin{align*}
\lambda_i^*g_i(x^*)\geq0, \hspace{0.5cm}i=1,..,m
\end{align*}

Vu le calcul précédent, on a pour valeur de $L(x^*,\lambda^*)$ :

In [10]:
print(L)

-1.4142665948363262


On vérifie d'abord les conditions **2** et **3**, étant les plus faciles à vérifier.
On calcule donc $g_i(x^*)$ : 

In [11]:
print('g1 en x optimal par la méthode de pénalité = ', g1(x))
print('g2 en x optimal par la méthode de pénalité = ', g2(x))
print('g3 en x optimal par la méthode de pénalité = ', g3(x))
print('g4 en x optimal par la méthode de pénalité = ', g4(x))

g1 en x optimal par la méthode de pénalité =  -0.6428401339438541
g2 en x optimal par la méthode de pénalité =  1.0845265457692932e-05
g3 en x optimal par la méthode de pénalité =  1.5318810241796736e-05
g4 en x optimal par la méthode de pénalité =  -1.0000153188102419


**Conclusion sur la condition 2):**

Directement, on a 
\begin{align*}
g_i(x^*)\ngeq0, \hspace{0.5cm} i=1,..,m
\end{align*}
puisque, par exemple, $g_1(x)< -0.5<0$

**Donc le point calculé n'est pas un point col.**

# Partie 2 - Méthode de barrière :

Le problème est le suivant : 

\begin{align*}
Min \hspace{0.1cm} -x_1-x_2+x_3
\\sous \hspace{0.1cm}contraintes
\\0 \leq x_3 \leq 2
\\x_1^3+x_3 \leq 2
\\x_1^2+x_2^2+x_3^2 \leq 2
\end{align*}

***Problème général avec la méthode de barrières :***

Contraintes d'inégalité : 
\begin{align*}
min f(x) 
\end{align*}
sous contraintes 
\begin{align*}
g_i(x) \leq 0,i=1,...,m
\end{align*}

Problème pénalisé avec les méthodes de barrières : 
\begin{align*}
min_x f(x)-\rho \sum_i log(-g_i(x))
\end{align*}

Condition d'optimalité : 
\begin{align*}
P(x,\rho)=\nabla f(x)-\rho \sum_i(1/g_i(x))\nabla g_i(x)=0
\end{align*}


Méthodes intérieures vu que $g_i(x)<0$

**Ici on a :** 
\begin{align*}
f(x)=-x_1 -x_2 +x_3\\
g_1(x)=-x_3\\
g_2(x)=x_3-2\\
g_3(x)=x_1^3+x_3-2\\
g_4(x)=x_1^2+x_2^2+x_3^2-2\\
\end{align*}

### **1)**On détermine la solution optimale de ce problème en utilisant une méthode de barrière: 
On définit dans un premier temps l'ensemble des fonctions du problème ainsi que leur gradients.

In [12]:
def f(x):
    return -x[0]-x[1]+x[2]
def g1(x):
    return -x[2]
def g2(x):
    return x[2]-2
def g3(x):
    return x[0]**2 +x[2]-2
def g4(x):
    return x[0]**2+x[1]**2+x[2]**2-2

def gradf(x):
    return np.array([-1,-1,1])
def gradg1(x):
    return np.array([0,0,-1])
def gradg2(x):
    return np.array([0,0,1])
def gradg3(x):
    return np.array([3*x[0]**2,0,1])
def gradg4(x):
    return np.array([2*x[0],2*x[1],2*x[2]])

def P(x):
    p1=(1/g1(x))*gradg1(x)
    p2=(1/g2(x))*gradg2(x)
    p3=(1/g3(x))*gradg3(x)
    p4=(1/g4(x))*gradg4(x)
    Sum=p1+p2+p3+p4
    return np.linalg.norm(gradf(x) - rho*Sum)

On applique la méthode de barrière :

In [13]:
epsilon = 0.00001
beta=0.6
x=np.array([1,0.2,0.7])
rho=0.7
tolerance=0.01
k=1 

while rho>epsilon:
    x=x
    while P(x)>tolerance:
        """
        Nous voulions dans un premier temps résoudre la minimisation de P de façon manuelle grâce à ce type de manoeuvres :
        
        
        On détermine une direction admissible :
        direction d du gradient:
        d=-np.gradient(P(x))
        (On pourrait aussi prendre le direction de Newton par exemple)
        On détermine un pas admissible :
        alpha=opt.line_search(P,gradP,x,d)
        On met à jour x : 
        x=x+alpha[0]*d
        
        Cependant, ces essais n'ayant pas abouties, 
        on résout la minimisation de P tout simplement avec la fonction minimize
        de la librairie scipy.optmize : 
        """
        x=opt.minimize(P,x).x        
    x=x
    rho=beta*rho
    k+=1
print('x : ', x)
print('valeur de f en x  : ' ,f(x))

x :  [9.99438305e-01 1.00054604e+00 1.53467605e-05]
valeur de f en x  :  -1.9999689973097867


**Nous avons donc trouvé une valeur de x solution de ce problème par la méthode de barrière.**
Comme pour la méthode de pénalité, on a choisi les valeurs initiales à dessein.

#### Vérification de la statisfaction des contraintes :

In [14]:
ineq2=x[0]**3+x[2]
ineq3=x[0]**2+x[1]**2+x[2]**2

Première inégalité : $0 \leq x_3 \leq 2$ Or, ici : $x_3=$

In [15]:
print(x[2])

1.534676046071751e-05


Deuxième inégalité : $x_1^3+x_3 \leq 2$ Or, ici, $x_1^3+x_3=$ 

In [16]:
print(ineq2)

0.998331207600565


Troisième inégalité : $x_1^2+x_2^2+x_3^2 \leq 2$ Or, ici, $x_1^2+x_2^2+x_3^2=$ 

In [17]:
print(ineq3)

1.9999693020363172


**On vérifie donc dans l'ensemble qu'il s'agit bien d'une méthode intérieure, puisque toutes les contraintes sont parfaitement satisfaites**

### 2) Estimation des valeurs des multiplicateurs de Lagrange : 
Etant donné la nature de la formule issue des conditions de Karush-Kuhn-Tucker et la condition d'optimalité, on peut approcher chaque multiplicateur de Lagrange $\lambda_i$ par la formule : 
\begin{align*}
\lambda_i = \frac{-\rho}{g_i(x)}
\end{align*}
Ainsi on pour multiplicateurs de Lagrange : 

In [18]:
lamb1=(-rho/g1(x))
lamb2=(-rho/g2(x))
lamb3=(-rho/g3(x))
lamb4=(-rho/g4(x))
print('1er Multiplicateur de Lagrange : ', lamb1)
print('2eme Multiplicateur de Lagrange : ', lamb2)
print('3eme Multiplicateur de Lagrange : ', lamb3)
print('4eme Multiplicateur de Lagrange : ', lamb4)

1er Multiplicateur de Lagrange :  0.6003559704044491
2eme Multiplicateur de Lagrange :  4.606794984168903e-06
3eme Multiplicateur de Lagrange :  9.203324488152282e-06
4eme Multiplicateur de Lagrange :  0.30013454195705325


#### Contraintes actives :
Une contrainte est active si son multiplicateur de Lagrange associé est non-nul. 

Ainsi, étant donné les valeurs des multiplicateurs de Lagrange ici, les contraintes associées aux valeurs non nulles des multiplicateurs de Lagrange sont actives, c'est-à-dire les contraintes associées à $\lambda_1$ et à $\lambda_4$ donc les contraintes suivantes : 

\begin{align*}
\\0 \leq x_3 
\\x_1^2+x_2^2+x_3^2 \leq 2
\end{align*}

En effet, étant donné $\lambda_2\leq10^{-5}$ et $\lambda_3\leq10^{-5}$, on fait l'approcimation qu'elles sont nulles, et donc inactives. 

### 3) Les conditions de complémentarité sont-elles satisfaites ? 

Les conditions de complémentarité sont satisfaites si et seulement si : 
\begin{align*}
\lambda_ig_i(x^*)=0
\end{align*}
pour i =1,..,m

Ici, étant donné les formules ayant permis d'obtenir l'approximation des multiplicateurs de Lagrange, on obtiendra toujours :  
\begin{align*}
\lambda_ig_i(x^*)=-\rho \hspace{0.5cm} \forall i
\end{align*}

Or, on a pour valeurs de $\rho$ :

In [19]:
print(rho)

9.213519268958691e-06


Ainsi, on a $\rho\leq10^{-5}$.
Donc puisque les valeurs de $x$ et de $\lambda_i$ et même de $g_i(x)$ dont approximées, ont peut se dire que 
\begin{align*}
\lambda_ig_i(x^*)\approx0\hspace{0.5cm} \forall i
\end{align*}
et donc que 
\begin{align*}
\lambda_ig_i(x^*)=0\hspace{0.5cm} \forall i
\end{align*}

**Ainsi, à une certaines approximation près, les conditions de complémentarité sont satisfaites.**